In [9]:
import json
import requests
import os
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
import re


In [2]:
spotify_client = os.environ.get('SPOTIFY_CLIENT')
spotify_secret = os.environ.get('SPOTIFY_SECRET')

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=spotify_client,  
                                               client_secret=spotify_secret,
                                               redirect_uri='http://localhost:8888/callback/',
                                               scope='user-read-currently-playing'))


In [3]:
current_song = sp.currently_playing(market='FR', additional_types='track')

print(current_song)

{'timestamp': 1605021520487, 'context': {'external_urls': {'spotify': 'https://open.spotify.com/playlist/2hd4estp6il1q391dhVTSA'}, 'href': 'https://api.spotify.com/v1/playlists/2hd4estp6il1q391dhVTSA', 'type': 'playlist', 'uri': 'spotify:playlist:2hd4estp6il1q391dhVTSA'}, 'progress_ms': 106949, 'item': {'album': {'album_type': 'single', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/360IAlyVv4PCEVjgyMZrxK'}, 'href': 'https://api.spotify.com/v1/artists/360IAlyVv4PCEVjgyMZrxK', 'id': '360IAlyVv4PCEVjgyMZrxK', 'name': 'Miguel', 'type': 'artist', 'uri': 'spotify:artist:360IAlyVv4PCEVjgyMZrxK'}], 'external_urls': {'spotify': 'https://open.spotify.com/album/4sJXyIfwduIWFISb4iCq2f'}, 'href': 'https://api.spotify.com/v1/albums/4sJXyIfwduIWFISb4iCq2f', 'id': '4sJXyIfwduIWFISb4iCq2f', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab67616d0000b273c4daf0fda7e16ba5b0f86830', 'width': 640}, {'height': 300, 'url': 'https://i.scdn.co/image/ab67616d00001e02c4daf

In [29]:
flat_df = pd.io.json.json_normalize(current_song)
flat_df

,timestamp,progress_ms,currently_playing_type,is_playing,context.external_urls.spotify,context.href,context.type,context.uri,item.album.album_type,item.album.artists,...,item.id,item.is_local,item.is_playable,item.name,item.popularity,item.preview_url,item.track_number,item.type,item.uri,actions.disallows.resuming
0,1605021520487,106949,track,True,https://open.spotify.com/playlist/2hd4estp6il1...,https://api.spotify.com/v1/playlists/2hd4estp6...,playlist,spotify:playlist:2hd4estp6il1q391dhVTSA,single,[{'external_urls': {'spotify': 'https://open.s...,...,3lB0GMiI5KxDbTOG8V3bOx,False,True,waves - Tame Impala Remix,68,https://p.scdn.co/mp3-preview/ae9a6446d61c4bd3...,1,track,spotify:track:3lB0GMiI5KxDbTOG8V3bOx,True


In [43]:
playlist = re.split(':', flat_df['context.uri'][0])[2]
track_name = flat_df['item.name'][0]
track_artist = flat_df['item.artists'][0][0]['name']

#flat_df.columns
#track_name
#playlist
track_artist

'Miguel'